# Sky-MoCa -- Simulated Annealing for a 3D Spin Lattice

## TODOS

* change B field to point into x direction
* don't pick uniformly at random but certain distribution (depending on orientation of neighbours)
* sweep linearly over lattice, not randomly

## Parameters & Initialization

In [ ]:
using HDF5
using StatsBase
using ProgressMeter

In [ ]:
srand(42);

### Type Definitions and Initialization

In [ ]:
type Schedule
    T::Vector{Float64} # the annealing schedule for the temperature
    B::Vector{Float64} # the annealing schedule for the magnetic field
    Ntherm::Vector{Int64} # number of sweeps for thermalization
    Nsweep::Vector{Int64} # number of sweeps at each stage of the schedule
    Nconfig::Vector{Int64} # number of configs at each stage of the schedule
    N::Int64 # number of stages in schedule
    os::Int64 # current index in the schedule
    function Schedule(T::Vector{Float64}, B::Vector{Float64}, Ntherm::Vector{Int64}, Nsweep::Vector{Int64}, Nconfig::Vector{Int64})
        N = length(T)+length(B)-1
        N == length(Nconfig) == length(Nsweep) || error("Invalid dimensions.")
        new(T, B, Ntherm, Nsweep, Nconfig, N, 1)
    end
    function Schedule(T::Vector{Float64}, B::Vector{Float64}; Ntherm::Int64=250, Nsweep::Int64=30, Nconfig::Int64=250)
        N = length(T)+length(B)-1
        Schedule(T, B, fill(Ntherm, N), fill(Nsweep, N), fill(Nconfig, N))
    end
end

In [ ]:
type Parameters
    Nx::Int64 # number of lattice points in the x direction
    Ny::Int64 # number of lattice points in the y direction
    Nz::Int64 # number of lattice points in the z direction
    Ntot::Int64 # total number of lattice points
    T::Float64 # the current temperature
    J::Float64 # ferromagnetic exchange
    Jp::Float64 # ferromagnetic exchange of correction term
    K::Float64 # DM interaction
    Kp::Float64 # DM interaction of correction term
    B::Float64 # magnetic field in 3 direction
    function Parameters(; Nx::Int64=30, Ny::Int64=30, Nz::Int64=30, T::Float64=0.0, J::Float64=1.0, K::Float64=tan(2pi/10.0), B::Float64=0.0)
        Ntot = Nx*Ny*Nz; Jp = J/16.0; Kp = K/8.0
        new(Nx, Ny, Nz, Ntot, T, J, Jp, K, Kp, B)
    end
    function Parameters(anneal::Schedule; Nx::Int64=30, Ny::Int64=30, Nz::Int64=30)
        Parameters(Nx=Nx, Ny=Ny, Nz=Nz, T=anneal.T[1], B=anneal.B[1])
    end
end

In [ ]:
type Monitor
    Nrej::Int64 # number of overall rejected moves
    Nacc::Int64 # number of overall accepted moves
    nacc::Int64 # number of accepted moves per config
    accRate::Vector{Float64} # evolution of acceptance rate
    Monitor(anneal::Schedule) = new(0, 0, 0, zeros(Float64, sum(anneal.Nconfig)))
end

In [ ]:
type Results
    E::Vector{Float64} # total energy
    Etmp::Float64 # current total energy
    M::Array{Float64,2} # total magnetization
    Mtmp::Vector{Float64} # current total magnetization
    T::Vector{Float64} # temperature
    B::Vector{Float64} # external magnetic field
    os::Int64 # current offset (monte carlo time)
    function Results(anneal::Schedule)
        N = sum(anneal.Nconfig)
        new(zeros(Float64, N), 0.0, zeros(Float64, 3, N), zeros(Float64, 3), zeros(Float64, N), zeros(Float64, N), 0)
    end
end

In [ ]:
@inline init(anneal::Schedule) = (Monitor(anneal), Results(anneal))

### Grid Initialization

In [ ]:
function uniformS2(dims::Tuple{Int64,Int64,Int64})
    phi = 2pi .* rand(dims)
    u = 2rand(dims) - 1.0
    squ = sqrt(1.0 - u.*u)
    x = squ .* cos(phi)
    y = squ .* sin(phi)
    res = zeros(3,dims...)
    # TODO: this should be doable with smart stacking and reshape
    for i=1:*(dims...)
        @inbounds res[:,i] = [x[i] y[i] u[i]]
    end
    res
end

@inline function uniformS2!(s::Vector{Float64})
    phi = 2pi*rand()
    u = 2rand() - 1.0
    squ = sqrt(1.0 - u*u)
    s[1:3] = [squ*cos(phi), squ*sin(phi), u]
    nothing
end

In [ ]:
function initRandomGrid(dims::Tuple{Int64,Int64,Int64})
    grid = uniformS2(dims)
    res.Etmp = totalEnergy(grid, openboundary=OPENBOUNDARY)
    res.Mtmp = totalMagnetization(grid)
    info("Initialized random grid.")
    grid
end

@inline initRandomGrid() = initRandomGrid((pars.Nx, pars.Ny, pars.Nz))

In [ ]:
function initGridFromFile(filename::ASCIIString)
    grid = Array(Float64, 3, pars.Nx, pars.Ny, pars.Nz)
    h5open(getPath(filename), "r") do file
        lastconfig = sort([parse(Int64, match(r"[0-9]+", dset).match) for dset in names(file["/configs"])])[end]
        grid = squeeze(file[string("/configs/configs_", lastconfig)][:,:,:,:,end],5)
    end
    size(grid) == (3, pars.Nx, pars.Ny, pars.Nz) || error("Dimension mismatch.")
    grid
end

In [ ]:
@inline idx(i::Int64, N::Int64) = mod(i-1,N)+1

## Hamiltonian

In [ ]:
@inline crossX(a::Vector{Float64}, b::Vector{Float64}) = -a[3]*b[2] + a[2]*b[3]
@inline crossY(a::Vector{Float64}, b::Vector{Float64}) =  a[3]*b[1] - a[1]*b[3]
@inline crossZ(a::Vector{Float64}, b::Vector{Float64}) = -a[2]*b[1] + a[1]*b[2]

In [ ]:
function DMterm(grid::Array{Float64,4}, i::Int64, j::Int64, k::Int64, os::Int64)
    crossX(grid[:,i,j,k], grid[:,idx(i+os,pars.Nx),j,k]) +
    crossY(grid[:,i,j,k], grid[:,i,idx(j+os,pars.Ny),k]) +
    crossZ(grid[:,i,j,k], grid[:,i,j,idx(k+os,pars.Nz)])
end

In [ ]:
function totalEnergy(grid::Array{Float64,4}; openboundary::Bool=false)
    openboundary || return totalEnergyOpen(grid)
    Nx = pars.Nx; Ny = pars.Ny; Nz = pars.Nz
    E = 0.0
    for k=1:Nz, j=1:Ny, i=1:Nx
        E += - pars.B * grid[3,i,j,k] -
        pars.J * dot(grid[:,i,j,k], grid[:,idx(i+1,Nx),j,k] + grid[:,i,idx(j+1,Ny),k] + grid[:,i,j,idx(k+1,Nz)]) +
        pars.Jp * dot(grid[:,i,j,k], grid[:,idx(i+2,Nx),j,k] + grid[:,i,idx(j+2,Ny),k] + grid[:,i,j,idx(k+2,Nz)]) -
        pars.K * DMterm(grid, i, j, k, 1) + pars.Kp * DMterm(grid, i, j, k, 2)
    end
    E
end

function totalEnergyOpen(grid::Array{Float64,4})
    Nx = pars.Nx; Ny = pars.Ny; Nz = pars.Nz
    E = 0.0
    for i=1:Nx
        for j=1:Ny
            for k=1:Nz-2
                E += -pars.B * grid[3,i,j,k] -
                pars.J * dot(grid[:,i,j,k], grid[:,idx(i+1,Nx),j,k] + grid[:,i,idx(j+1,Ny),k] + grid[:,i,j,idx(k+1,Nz)]) +
                pars.Jp * dot(grid[:,i,j,k], grid[:,idx(i+2,Nx),j,k] + grid[:,i,idx(j+2,Ny),k] + grid[:,i,j,idx(k+2,Nz)]) -
                pars.K * DMterm(grid, i, j, k, 1) + pars.Kp * DMterm(grid, i, j, k, 2)
            end
            E += -pars.J * dot(grid[:,i,j,Nz-1], grid[:,idx(i+1,Nx),j,Nz-1] + grid[:,i,idx(j+1,Ny),Nz-1] + grid[:,i,j,Nz]) +
            pars.Jp * dot(grid[:,i,j,Nz-1], grid[:,idx(i+2,Nx),j,Nz-1] + grid[:,i,idx(j+2,Ny),Nz-1]) -
            pars.K * DMterm(grid, i, j, Nz-1, 1) +
            pars.Kp * (crossX(grid[:,i,j,Nz-1], grid[:,idx(i+2,Nx),j,Nz-1]) + crossY(grid[:,i,j,Nz-1], grid[:,i,idx(j+2,Ny),Nz-1]))

            E += -pars.J * dot(grid[:,i,j,Nz], grid[:,idx(i+1,Nx),j,Nz] + grid[:,i,idx(j+1,Ny),Nz]) +
            pars.Jp * dot(grid[:,i,j,Nz], grid[:,idx(i+2,Nx),j,Nz] + grid[:,i,idx(j+2,Ny),Nz]) -
            pars.K * (crossX(grid[:,i,j,Nz], grid[:,idx(i+1,Nx),j,Nz]) + crossY(grid[:,i,j,Nz], grid[:,i,idx(j+1,Ny),Nz])) +
            pars.Kp * (crossX(grid[:,i,j,Nz], grid[:,idx(i+2,Nx),j,Nz]) + crossY(grid[:,i,j,Nz], grid[:,i,idx(j+2,Ny),Nz]))
        end
    end
    E
end

In [ ]:
function totalMagnetization(grid::Array{Float64,4})
    M = zeros(Float64, 3)
    for i=1:pars.Ntot
        @inbounds M += grid[:,i]
    end
    M
end

In [ ]:
function energyChange(grid::Array{Float64,4}, snew::Vector{Float64}, i::Int64, j::Int64, k::Int64; openboundary::Bool=false)
    openboundary || return energyChangeOpen(grid, snew, i, j, k)
    const Nx = pars.Nx::Int64
    const Ny = pars.Ny::Int64
    const Nz = pars.Nz::Int64
    im1 = idx(i-1,Nx); im2 = idx(i-2,Nx); ip1 = idx(i+1,Nx); ip2 = idx(i+2,Nx)
    jm1 = idx(j-1,Ny); jm2 = idx(j-2,Ny); jp1 = idx(j+1,Ny); jp2 = idx(j+2,Ny)
    km1 = idx(k-1,Nz); km2 = idx(k-2,Nz); kp1 = idx(k+1,Nz); kp2 = idx(k+2,Nz)
    sdiff = snew - grid[:,i,j,k]
    -pars.B::Float64 * sdiff[3] -
    pars.J::Float64  * dot(sdiff, grid[:,im1,j,k] + grid[:,ip1,j,k] + grid[:,i,jm1,k] + grid[:,i,jp1,k] + grid[:,i,j,km1] + grid[:,i,j,kp1]) +
    pars.Jp::Float64 * dot(sdiff, grid[:,im2,j,k] + grid[:,ip2,j,k] + grid[:,i,jm2,k] + grid[:,i,jp2,k] + grid[:,i,j,km2] + grid[:,i,j,kp2]) -
    pars.K::Float64  * (crossX(sdiff, grid[:,ip1,j,k] - grid[:,im1,j,k]) +
                        crossY(sdiff, grid[:,i,jp1,k] - grid[:,i,jm1,k]) +
                        crossZ(sdiff, grid[:,i,j,kp1] - grid[:,i,j,km1])) +
    pars.Kp::Float64 * (crossX(sdiff, grid[:,ip2,j,k] - grid[:,im2,j,k]) +
                        crossY(sdiff, grid[:,i,jp2,k] - grid[:,i,jm2,k]) +
                        crossZ(sdiff, grid[:,i,j,kp2] - grid[:,i,j,km2]))
end

function energyChangeOpen(grid::Array{Float64,4}, snew::Vector{Float64}, i::Int64, j::Int64, k::Int64)
    const Nx = pars.Nx::Int64
    const Ny = pars.Ny::Int64
    const Nz = pars.Nz::Int64
    im1 = grid[:,idx(i-1,Nx),j,k]; im2 = grid[:,idx(i-2,Nx),j,k]
    ip1 = grid[:,idx(i+1,Nx),j,k]; ip2 = grid[:,idx(i+2,Nx),j,k]
    jm1 = grid[:,i,idx(j-1,Ny),k]; jm2 = grid[:,i,idx(j-2,Ny),k]
    jp1 = grid[:,i,idx(j+1,Ny),k]; jp2 = grid[:,i,idx(j+2,Ny),k]
    km1 = k == 1 ? zeros(3) : grid[:,i,j,idx(k-1,Nz)]; km2 = k <= 2 ? zeros(3) : grid[:,i,j,idx(k-2,Nz)]
    kp1 = k == Nz ? zeros(3) : grid[:,i,j,idx(k+1,Nz)]; kp2 = k >= Nz-1 ? zeros(3) : grid[:,i,j,idx(k+2,Nz)]
    sdiff = snew - grid[:,i,j,k]
    -pars.B::Float64 * sdiff[3] -
    pars.J::Float64  * dot(sdiff, im1 + ip1 + jm1 + jp1 + km1 + kp1) +
    pars.Jp::Float64 * dot(sdiff, im2 + ip2 + jm2 + jp2 + km2 + kp2) -
    pars.K::Float64  * (crossX(sdiff, ip1 - im1) + crossY(sdiff, jp1 - jm1) + crossZ(sdiff, kp1 - km1)) +
    pars.Kp::Float64 * (crossX(sdiff, ip2 - im2) + crossY(sdiff, jp2 - jm2) + crossZ(sdiff, kp2 - km2))
end

## Simulated Annealing

In [ ]:
function scheduleNext(anneal::Schedule)
    anneal.os += 1
    anneal.os > length(anneal.T) ? (pars.B = anneal.B[anneal.os - length(anneal.T)]) : (pars.T = anneal.T[anneal.os])
end

In [ ]:
@inline acceptProb(del::Float64, T::Float64) = exp(-del/T)

In [ ]:
function acceptMove!(grid::Array{Float64,4}, snew::Vector{Float64}, dE::Float64, i::Int64, j::Int64, k::Int64)
    mon.Nacc::Int64 += 1
    mon.nacc::Int64 += 1
    res.Etmp::Float64 += dE
    res.Mtmp::Vector{Float64} += snew - grid[:,i,j,k]
    grid[:,i,j,k] = snew
end

In [ ]:
function update!(grid::Array{Float64,4}, snew::Vector{Float64})
    i = rand(1:pars.Nx::Int64)
    j = rand(1:pars.Ny::Int64)
    k = rand(1:pars.Nz::Int64)
    uniformS2!(snew)
    dE = energyChange(grid, snew, i, j, k, openboundary=OPENBOUNDARY)
    if dE < 0.0 || acceptProb(dE, pars.T::Float64) >= rand()
        acceptMove!(grid, snew, dE, i, j, k)
    else
        mon.Nrej::Int64 += 1
    end
end

In [ ]:
@inline function sweep!(grid::Array{Float64,4}, snew::Vector{Float64})
    for i=1:pars.Ntot::Int64
        update!(grid, snew)
    end
end

In [ ]:
function nextconfig!(grid::Array{Float64,4})
    snew = zeros(Float64, 3)
    for j=1:anneal.Nsweep[anneal.os]::Int64
        sweep!(grid, snew)
    end
end

In [ ]:
function thermalize!(grid::Array{Float64,4}, anneal::Schedule)
    info("Start thermalizing...")
    snew = zeros(Float64, 3)
    @showprogress 1 "Thermalizing..." for i=1:anneal.Ntherm[anneal.os]::Int64
        sweep!(grid, snew)
    end
    info("Finished thermalizing.")
    grid
end

function thermalize(anneal::Schedule)
    grid = initRandomGrid()
    thermalize!(grid, anneal)
end

In [ ]:
function runstage!(grid::Array{Float64,4}, file::HDF5File, anneal::Schedule)
    conf, avg = initOutput(file, anneal)
    snapshot(grid, conf, avg, anneal)
    @showprogress 1 "Stage $(anneal.os)..." for i=1:anneal.Nconfig[anneal.os]::Int64-1
        nextconfig!(grid)
        snapshot(grid, conf, avg, anneal)
    end
    stageOutput(file, avg, anneal)
    info("Finished stage $(anneal.os).")
    scheduleNext(anneal)
end

In [ ]:
function run!(grid::Array{Float64,4}, filename::ASCIIString, anneal::Schedule)
    info("Start run with $(anneal.N) stages...")
    thermalize!(grid, anneal)
    h5open(getPath(filename), "w") do file
        for i = 1:anneal.N::Int64
            runstage!(grid, file, anneal)
        end
        finalOutput(file, anneal)
    end
    info("Finished all stages.")
end

function run(filename::ASCIIString, anneal::Schedule; oldfile::ASCIIString="")
    if oldfile == ""
        info("Using new random grid.")
        grid = initRandomGrid()
    else
        info(string("Continue from last config of ", oldfile))
        grid = initGridFromFile(oldfile)
    end
    run!(grid, filename, anneal)
end

## I/O

In [ ]:
function snapshot(grid::Array{Float64,4}, conf::HDF5Dataset, avg::HDF5Dataset, anneal::Schedule)
    res.os::Int64 += 1
    res.E[res.os] = res.Etmp::Float64
    res.M[:,res.os] = res.Mtmp::Vector{Float64}
    res.T[res.os] = pars.T::Float64
    res.B[res.os] = pars.B::Float64
    mon.accRate[res.os] = mon.nacc::Int64 / (pars.Ntot::Int64 * anneal.Nsweep[anneal.os]::Int64)
    mon.nacc::Int64 = 0
    appendConfig(grid, res.os-sum(anneal.Nconfig[1:anneal.os-1]), conf)
    addConfig(grid, avg)
end

In [ ]:
function initOutput(file::HDF5File, anneal::Schedule)
    conf = d_create(file, "configs/configs_$(anneal.os)", datatype(Float64), dataspace(3, pars.Nx, pars.Ny, pars.Nz, anneal.Nconfig[anneal.os]), "chunk", (3, pars.Nx, pars.Ny, pars.Nz, 1))
    avg = d_create(file, "avgs/avg_$(anneal.os)", datatype(Float64), dataspace(3, pars.Nx, pars.Ny, pars.Nz))
    info("Created output file for stage $(anneal.os).")
    (conf, avg)
end

In [ ]:
function stageOutput(file::HDF5File, avg::HDF5Dataset, anneal::Schedule)
    averageConfigs!(avg, anneal)
    i = sum(anneal.Nconfig[1:anneal.os-1])+1
    f = i + anneal.Nconfig[anneal.os] - 1
    writeResultAndError(file, res.E[i:f], "res/E_avg_$(anneal.os)")
    writeResultAndError(file, getX(res.M[:,i:f]), "res/Mx_avg_$(anneal.os)")
    writeResultAndError(file, getZ(res.M[:,i:f]), "res/My_avg_$(anneal.os)")
    writeResultAndError(file, getZ(res.M[:,i:f]), "res/Mz_avg_$(anneal.os)")
    writeResultAndError(file, res.E[i:f], "res/specific_heat_$(anneal.os)", fun=specificHeat)
    writeResultAndError(file, getZ(res.M[:,i:f]), "res/susceptibility_$(anneal.os)", fun=susceptibility)    
end

In [ ]:
function finalOutput(file::HDF5File, anneal::Schedule)
    file["energy"] = res.E
    file["magnetization"] = res.M
    file["temperature"] = res.T
    file["magnetic_field"]=res.B
    file["acceptancerate"] = mon.accRate
    writeParameters(file, anneal)
end

In [ ]:
function writeParameters(file::HDF5File, anneal::Schedule)
    file["parameters/Nx"] = pars.Nx
    file["parameters/Ny"] = pars.Ny
    file["parameters/Nz"] = pars.Nz
    file["parameters/Ntherm"] = anneal.Ntherm
    file["parameters/Nsweep"] = anneal.Nsweep
    file["parameters/Nconfig"] = anneal.Nconfig
    file["parameters/J"] = pars.J
    file["parameters/Jp"] = pars.Jp
    file["parameters/K"] = pars.K
    file["parameters/Kp"] = pars.Kp
end

In [ ]:
@inline function writeResultAndError(file::HDF5File, res::Float64, err::Float64, dset::ASCIIString)
    file[dset] = [res, err]
end

function writeResultAndError(file::HDF5File, f::Vector{Float64}, dset::ASCIIString; fun::Function=mean)
    res = fun(f)
    err = sqrt(jackknife(fun, f))
    writeResultAndError(file, res, err, dset)
end

In [ ]:
@inline function averageConfigs!(avg::HDF5Dataset, anneal::Schedule)
    avg[:,:,:,:] /= anneal.Nconfig[anneal.os]
end

In [ ]:
@inline function appendConfig(grid::Array{Float64,4}, os::Int64, conf::HDF5Dataset)
    conf[:,:,:,:,os] = grid
end

In [ ]:
@inline function addConfig(grid::Array{Float64,4}, avg::HDF5Dataset)
    avg[:,:,:,:] += grid
end

In [ ]:
@inline getPath(filename::ASCIIString; ext::ASCIIString="h5") = string("../../data/", filename, ".", ext)

In [ ]:
function loadData(filename::ASCIIString, dset::ASCIIString; group=nothing)
    h5open(getPath(filename), "r") do file
        if group == nothing
            idx = dset
        else
            idx = string(group, "/", dset)
        end
        data = read(file[idx])
    end
end

In [ ]:
function loadParameters(filename::ASCIIString)
    h5open(getPath(filename), "r") do f
        pars.Nx = read(f["parameters/Nx"])
        pars.Ny = read(f["parameters/Ny"])
        pars.Nz = read(f["parameters/Nz"])
        pars.Ntot = pars.Nx * pars.Ny * pars.Nz
        anneal.Nconfig = read(f["parameters/Nconfig"])
        anneal.Nsweep = read(f["parameters/Nsweep"])
        anneal.Ntherm = read(f["parameters/Ntherm"])
        pars.T = read(f["temperature"])[1]
    end
end

## Postprocessing

### Basic Statistics

In [ ]:
function autocorr(f::Vector{Float64})
    mean, var = mean_and_var(f)
    N = length(f)
    fm = f - mean
    auto = zeros(Float64, N)
    for t=0:N-1
        tmp = 0.0
        for i=1:N-t
            tmp += fm[i]*fm[i+t]
        end
        auto[t+1] = tmp / ((N-t)*var)
    end
    auto
end

In [ ]:
function intAutoTime(f::Vector{Float64}, N::Int64; isautocorr=false)
    isautocorr || (f = autocorr(f))
    0.5 + sum(f[2:N])
end

@inline intAutoTime(f::Vector{Float64}; isautocorr=false) = intAutoTime(f, length(f), isautocorr=isautocorr)

In [ ]:
function jackknife(obs::Function, f::Vector{Float64})
    N = length(f)
    jack = [obs([f[1:i]; f[i+2:N]]) for i=0:N-1]
    (N-1) * mean( (jack - mean(jack)).^2 )
end

@inline jackknife(f::Vector{Float64}) = jackknife(mean, f)

### Skyrmion Specifics

In [ ]:
@inline getX(grid::Array{Float64}) = squeeze(grid[1,:],1)
@inline getY(grid::Array{Float64}) = squeeze(grid[2,:],1)
@inline getZ(grid::Array{Float64}) = squeeze(grid[3,:],1)
@inline getX(grid::Array{Float64,4}) = squeeze(grid[1,:,:,:],1)
@inline getY(grid::Array{Float64,4}) = squeeze(grid[2,:,:,:],1)
@inline getZ(grid::Array{Float64,4}) = squeeze(grid[3,:,:,:],1)

In [ ]:
function drawProj(f::Array{Float64,3}, dir::Int64; name::ASCIIString="proj", saveimg=false)
    proj = squeeze(sum(f, dir), dir)
    img = grayim( (proj/maximum(proj[:])).^6 )
    saveimg && save(getPath(string(name, "_", dir), extension="png"), img)
    img
end

function drawProj(f::Array{Float64,3}; name::ASCIIString="proj", save=false)
    [drawProj(f, dir, name=name, save=save) for dir=1:3]
end

In [ ]:
@inline rotatehalf(N::Int64) = vcat(fld(N,2)+1:N, 1:fld(N,2))

In [ ]:
function braggIntensity(grid::Array{Float64,4})
    Sk = fftVF(grid)
    [norm(Sk[:,i,j,k])::Float64 for i=rotatehalf(pars.Nx), j=rotatehalf(pars.Ny), k=rotatehalf(pars.Nz)]
end

In [ ]:
function fftVF(grid::Array{Float64,4})
    Sx = fft3(getX(grid))
    Sy = fft3(getY(grid))
    Sz = fft3(getZ(grid))
    S = zeros(Complex{Float64}, size(grid))
    # TODO: this should be doable with smart stacking and reshape
    for i=1:pars.Ntot::Int64
        @inbounds S[:,i] = [Sx[i] Sy[i] Sz[i]]
    end
    S
end

In [ ]:
function fft3(f::Array{Float64,3})
    FFTW.set_num_threads(4)
    fft(f - mean(f[:]))
end

In [ ]:
@inline specificHeat(f::Vector{Float64}) = var(f)/(pars.Ntot * pars.T^2)

In [ ]:
@inline susceptibility(f::Vector{Float64}) = var(f)/(pars.Ntot * pars.T)

## Run the Simulation

In [ ]:
const OPENBOUNDARY = true
anneal = Schedule([0.7], collect(0.1495:-.0005:0.12), fill(250,60), fill(30,60), fill(250,60))
pars = Parameters(anneal, Nx=42, Ny=42, Nz=30)
mon, res = init(anneal);
pars.T, pars.B, (pars.Nx, pars.Ny, pars.Nz), anneal.Ntherm, anneal.Nconfig, anneal.Nsweep

In [ ]:
@time run("Ntot$(pars.Ntot)", anneal);
# @time run("Ntot$(pars.Ntot)", anneal, oldfile="/scratch/users/nki/magnetism/anneal/Ntot52920");

## Analyse the Results

### First Plots of a Single Run

In [ ]:
using PyPlot

In [ ]:
tMC = 1:sum(anneal.Nconfig)
plot(res.E)
figure()
plot(tMC, getX(res.M), tMC, getY(res.M), tMC, getZ(res.M))

In [ ]:
Iauto = tMC-1
plot(autocor(res.E, Iauto))
figure()
plot(tMC, autocor(getX(res.M), Iauto),tMC, autocor(getY(res.M), Iauto),tMC, autocor(getZ(res.M), Iauto))

In [ ]:
plot(mon.accRate)

### Bragg Intensity

In [ ]:
using Images

In [ ]:
name = "focused/Ntot27000_T0.8_B0.1_cmpct"
avg = loadData(name, "avg");
bragg = braggIntensity(avg);

In [ ]:
drawProj(bragg, 3, saveimg=false)

### Combine Multiple Runs

In [ ]:
function plotResultT(Ts, Bs, dset)
    data = [loadData("focused/Ntot27000_T$(T)_B$(B)_cmpct", dset) for T in Ts, B in Bs]
    fig,ax = PyPlot.subplots()
    for (i,B) in enumerate(Bs)
        val = [data[j,i][1] for j in eachindex(Ts)]
        err = [data[j,i][2] for j in eachindex(Ts)]
        ax[:errorbar](Ts[5:end], val[5:end] + (i-1) * 0.5, yerr=err[5:end], fmt="o", label="B=$(B)")
    end
    ax[:legend](loc="upper right")
end

function plotResultB(Ts, Bs, dset)
    data = [loadData("focused/Ntot27000_T$(T)_B$(B)_cmpct", dset) for T in Ts, B in Bs]
    fig,ax = PyPlot.subplots()
    for (i,T) in enumerate(Ts)
        val = [data[i,j][1] for j in eachindex(Bs)]
        err = [data[i,j][2] for j in eachindex(Bs)]
        ax[:errorbar](Bs[:], val[:], yerr=err[:], fmt="o", label="T=$(T)")
    end
    ax[:legend](loc="upper right")
end

In [ ]:
Ts = 0.5:0.05:1.5
Bs = [0.0 0.01 0.02 0.05 0.1 0.15 0.2 0.25 0.3 0.35 0.4]
plotResult(Ts, Bs, "specific_heat")

In [ ]:
plotResult(Ts, Bs, "susceptibility")

## Tests

In [ ]:
using Base.Test

In [ ]:
function testuniformS2()
    N = 50
    dims = (N, N, N)
    grid = uniformS2(dims);
    for i=1:*(dims...)
        @test norm(grid[:,i]) ≈ 1.0
    end
    println("finished test: uniform points on S2")
end

In [ ]:
function test()
    testuniformS2()
    println("If you do not see any errors, all tests passed.")
end